In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/tmdb-movie-metadata/tmdb_5000_movies.csv
/kaggle/input/tmdb-movie-metadata/tmdb_5000_credits.csv
/kaggle/input/tmdbids/tmdbIds_movieLense.csv


In [4]:
import requests
import json

api_key = "78581c2472ba1134bd2f45d55d9486b6"
url = "https://api.themoviedb.org/3/trending/all/day"

# Send initial request to get the total number of pages
params = {"api_key": api_key, "page": 1}
response = requests.get(url, params=params)

if response.status_code == 200:
    total_pages = json.loads(response.text)["total_pages"]
    movies = []

    # Send requests for each page and concatenate the results
    for page in range(1, total_pages + 1):
        params["page"] = page
        response = requests.get(url, params=params)
        page_data = json.loads(response.text)

        if "results" in page_data:
            movies.extend(page_data["results"])
        else:
            print("Error: No results found in page", page)

    # Store the results in a file
    with open("popular_movies.json", "w") as f:
        json.dump(movies, f)
else:
    print("Error:", response.status_code, response.text)

In [28]:
import requests
import json

api_key = "78581c2472ba1134bd2f45d55d9486b6"
url = "https://api.themoviedb.org/3/movie/{movie_id}?api_key={api_key}&language=en-US"

movie_ids = li  # list of movie IDs to fetch data for

movie_data = []  # dictionary to store movie data
for movie_id in movie_ids:
    # make API call for each movie ID
    response = requests.get(url.format(movie_id=movie_id, api_key=api_key))
    if response.status_code != 200:
        # handle API error
        continue
    else:
        # extract JSON data from response
         movie_data.append(response.json())
with open("movielens1.json", "w") as f:
    json.dump(movie_data, f)
#print(json.dumps(movie_data, indent=4))

In [ ]:
df = pd.merge()

In [29]:
import json
import pandas as pd

with open('movielens1.json', 'r') as f:
    data = json.load(f)

movie_lens = pd.json_normalize(data)

In [34]:
movie_lens.to_csv("tmdb_data.csv")

In [32]:
movie_lens.shape

(26745, 29)

In [ ]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Embedding, Flatten, Dot, Concatenate

# Load the datasets
movies_df = pd.read_csv('movies.csv')
ratings_df = pd.read_csv('ratings.csv')

# Encode user and movie IDs
user_encoder = LabelEncoder()
movies_encoder = LabelEncoder()
ratings_df['user_id'] = user_encoder.fit_transform(ratings_df['user_id'].values)
ratings_df['movie_id'] = movies_encoder.fit_transform(ratings_df['movie_id'].values)

# Define the neural network architecture
num_users = ratings_df['user_id'].nunique()
num_movies = ratings_df['movie_id'].nunique()
embedding_size = 32

user_input = Input(shape=[1])
user_embedding = Embedding(num_users, embedding_size)(user_input)
user_flat = Flatten()(user_embedding)

movie_input = Input(shape=[1])
movie_embedding = Embedding(num_movies, embedding_size)(movie_input)
movie_flat = Flatten()(movie_embedding)

dot = Dot(axes=1)([user_flat, movie_flat])

# Optional: add metadata features to the neural network
genre_input = Input(shape=[1])
genre_embedding = Embedding(num_movies, embedding_size)(genre_input)
genre_flat = Flatten()(genre_embedding)

concat = Concatenate()([dot, genre_flat])

# Define the model and compile it
model = Model(inputs=[user_input, movie_input, genre_input], outputs=concat)
model.compile(loss='mse', optimizer='adam')

# Train the model on the ratings data
history = model.fit(
    x=[ratings_df['user_id'].values, ratings_df['movie_id'].values, movies_df['genre_ids'].apply(lambda x: x[0]).values], 
    y=ratings_df['rating'].values, 
    batch_size=64, 
    epochs=10,
    validation_split=0.1
)

# Use the model to predict ratings for new users and movies
user_id = 5
movie_id = 100
genre_id = 27
predicted_rating = model.predict([user_encoder.transform([user_id]), movies_encoder.transform([movie_id]), [genre_id]])

In [4]:
tmdbids = pd.read_csv("/kaggle/input/tmdbids/tmdbIds_movieLense.csv")

In [5]:
tmdbids.columns

Index(['Unnamed: 0', 'tmdbId'], dtype='object')

In [6]:
li = tmdbids['tmdbId']

In [7]:
li = list(li)

In [5]:
import json
import pandas as pd

with open('popular_movies.json', 'r') as f:
    data = json.load(f)

df = pd.json_normalize(data)

In [6]:
df1 = df[~df.title.isnull()]

In [7]:
df1 = df[~df.title.isnull()]

In [8]:
df1[df1['backdrop_path'].isnull()]

,adult,backdrop_path,id,title,original_language,original_title,overview,poster_path,media_type,genre_ids,...,vote_average,vote_count,name,original_name,first_air_date,origin_country,gender,known_for_department,profile_path,known_for
399,False,None,1104771,My House,en,My House,Carla lives life locked inside her house with ...,/1WRDMZr1JtjjIGYsRQzyqBw7dCf.jpg,movie,"[18, 9648, 53, 27]",...,4.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
548,False,None,1109433,Who Murdered Meredith Kercher,en,Who Murdered Meredith Kercher,"Italy, 2007. 21-year-old English exchange stud...",/40R5cvvogrdGm7L603xZB7B2bId.jpg,movie,[],...,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
802,False,None,1077321,Drunken Fist,zh,苏乞儿之武功盖世,"In the last years of the Qing Dynasty, Su Qier...",/pOHSYM8LUHc370q0QzR5tcfa0sR.jpg,movie,[28],...,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
989,False,None,1119212,Mexican Connection,en,Mexican Connection,Mexican Connection (2023) Story of an Italian...,/oX48pMF7vZuqlNDy1KH771elsnz.jpg,movie,[],...,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1124,False,None,353725,Our Loved Ones,fr,Les êtres chers,"1978. In a small village in Bas-St-Laurent, Gu...",/rFZXGb1sHBK7S3ad2lZ3ukcm9Uw.jpg,movie,[18],...,6.9,9.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19912,False,None,118417,7 seX 7,hr,7 seX 7,An Croatian erotic anthology of seven short st...,/aeENu0H0yeh1qRuyJMU7WJ81Edq.jpg,movie,"[18, 35]",...,4.3,14.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
19923,False,None,1041323,Lullaby,hy,Колыбельная,The film is the result of a charity workshop f...,None,movie,[16],...,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
19932,False,None,777200,Snake: Fall of a City,zh,沙蟒围城,"Racer Lin Chuang admired his rival Liang Zi, a...",/eus82zsOOXMd4GnjtHAXd4VYWcQ.jpg,movie,[18],...,1.5,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
19952,False,None,1023157,Europe‘s Largest Desert,en,Europe‘s Largest Desert,"Near the cold Pyrenees of Iberia, surrounded b...",/3zVNtkSpbYfGIBYYznWDmCHNzkc.jpg,movie,[99],...,6.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [9]:
df1.columns

Index(['adult', 'backdrop_path', 'id', 'title', 'original_language',
       'original_title', 'overview', 'poster_path', 'media_type', 'genre_ids',
       'popularity', 'release_date', 'video', 'vote_average', 'vote_count',
       'name', 'original_name', 'first_air_date', 'origin_country', 'gender',
       'known_for_department', 'profile_path', 'known_for'],
      dtype='object')

In [10]:
df1.drop(["poster_path","original_title","media_type","backdrop_path","video","name",'original_name', 'first_air_date', 'origin_country', 'gender',
       'known_for_department', 'profile_path', 'known_for'],axis=1,inplace=True)

/opt/conda/lib/python3.7/site-packages/pandas/core/frame.py:4913: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [11]:
import pandas as pd
import numpy as np

In [12]:
movies = pd.read_csv('/kaggle/input/tmdb-movie-metadata/tmdb_5000_movies.csv')
credits = pd.read_csv('/kaggle/input/tmdb-movie-metadata/tmdb_5000_credits.csv')

In [13]:
movies['year'] = pd.to_datetime(movies['release_date'], errors='coerce').apply(lambda x: str(x).split('-')[0] if x != np.nan else np.nan)

In [14]:
movies.columns

Index(['budget', 'genres', 'homepage', 'id', 'keywords', 'original_language',
       'original_title', 'overview', 'popularity', 'production_companies',
       'production_countries', 'release_date', 'revenue', 'runtime',
       'spoken_languages', 'status', 'tagline', 'title', 'vote_average',
       'vote_count', 'year'],
      dtype='object')

In [15]:
movies = movies[['id', 'title', 'overview', 'genres', 'keywords','vote_average','vote_count','year']]

In [16]:
movies.isnull().sum()

id              0
title           0
overview        3
genres          0
keywords        0
vote_average    0
vote_count      0
year            0
dtype: int64

In [17]:
movies.dropna(inplace = True)

In [18]:
df2 = movies.copy()

In [19]:
df2.head()

,id,title,overview,genres,keywords,vote_average,vote_count,year
0,19995,Avatar,"In the 22nd century, a paraplegic Marine is di...","[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...","[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...",7.2,11800,2009
1,285,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...","[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""...","[{""id"": 270, ""name"": ""ocean""}, {""id"": 726, ""na...",6.9,4500,2007
2,206647,Spectre,A cryptic message from Bond’s past sends him o...,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...","[{""id"": 470, ""name"": ""spy""}, {""id"": 818, ""name...",6.3,4466,2015
3,49026,The Dark Knight Rises,Following the death of District Attorney Harve...,"[{""id"": 28, ""name"": ""Action""}, {""id"": 80, ""nam...","[{""id"": 849, ""name"": ""dc comics""}, {""id"": 853,...",7.6,9106,2012
4,49529,John Carter,"John Carter is a war-weary, former military ca...","[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...","[{""id"": 818, ""name"": ""based on novel""}, {""id"":...",6.1,2124,2012


In [20]:
df2.overview[0]

'In the 22nd century, a paraplegic Marine is dispatched to the moon Pandora on a unique mission, but becomes torn between following orders and protecting an alien civilization.'

In [21]:
import json
def genres(value):
    output_value = []
    json_data = json.loads(value)
    for item in json_data:
        output_value.append(item['name'])
    return output_value

In [22]:
import json
def genres_1(value):
    output_value = []
    json_data = json.loads(value)
    for item in json_data:
        output_value.append(item['id'])
    return output_value

In [23]:
movies['genre_id'] = movies.genres.apply(lambda x :genres_1(x))

In [24]:
movies.genres[:50]

0     [{"id": 28, "name": "Action"}, {"id": 12, "nam...
1     [{"id": 12, "name": "Adventure"}, {"id": 14, "...
2     [{"id": 28, "name": "Action"}, {"id": 12, "nam...
3     [{"id": 28, "name": "Action"}, {"id": 80, "nam...
4     [{"id": 28, "name": "Action"}, {"id": 12, "nam...
5     [{"id": 14, "name": "Fantasy"}, {"id": 28, "na...
6     [{"id": 16, "name": "Animation"}, {"id": 10751...
7     [{"id": 28, "name": "Action"}, {"id": 12, "nam...
8     [{"id": 12, "name": "Adventure"}, {"id": 14, "...
9     [{"id": 28, "name": "Action"}, {"id": 12, "nam...
10    [{"id": 12, "name": "Adventure"}, {"id": 14, "...
11    [{"id": 12, "name": "Adventure"}, {"id": 28, "...
12    [{"id": 12, "name": "Adventure"}, {"id": 14, "...
13    [{"id": 28, "name": "Action"}, {"id": 12, "nam...
14    [{"id": 28, "name": "Action"}, {"id": 12, "nam...
15    [{"id": 12, "name": "Adventure"}, {"id": 10751...
16    [{"id": 878, "name": "Science Fiction"}, {"id"...
17    [{"id": 12, "name": "Adventure"}, {"id": 2

In [25]:
movies['genre_id']

0            [28, 12, 14, 878]
1                 [12, 14, 28]
2                 [28, 12, 80]
3             [28, 80, 18, 53]
4                [28, 12, 878]
                 ...          
4798              [28, 80, 53]
4799               [35, 10749]
4800    [35, 18, 10749, 10770]
4801                        []
4802                      [99]
Name: genre_id, Length: 4800, dtype: object

In [26]:
df2['keywords'][0]

'[{"id": 1463, "name": "culture clash"}, {"id": 2964, "name": "future"}, {"id": 3386, "name": "space war"}, {"id": 3388, "name": "space colony"}, {"id": 3679, "name": "society"}, {"id": 3801, "name": "space travel"}, {"id": 9685, "name": "futuristic"}, {"id": 9840, "name": "romance"}, {"id": 9882, "name": "space"}, {"id": 9951, "name": "alien"}, {"id": 10148, "name": "tribe"}, {"id": 10158, "name": "alien planet"}, {"id": 10987, "name": "cgi"}, {"id": 11399, "name": "marine"}, {"id": 13065, "name": "soldier"}, {"id": 14643, "name": "battle"}, {"id": 14720, "name": "love affair"}, {"id": 165431, "name": "anti war"}, {"id": 193554, "name": "power relations"}, {"id": 206690, "name": "mind and soul"}, {"id": 209714, "name": "3d"}]'

In [27]:
df2['genres'] = df2.genres.apply(lambda x : genres(x))

In [28]:
df2['keywords'] = df2.keywords.apply(lambda x: genres(x))

In [29]:
df2['keywords']

0       [culture clash, future, space war, space colon...
1       [ocean, drug abuse, exotic island, east india ...
2       [spy, based on novel, secret agent, sequel, mi...
3       [dc comics, crime fighter, terrorist, secret i...
4       [based on novel, mars, medallion, space travel...
                              ...                        
4798    [united states–mexico barrier, legs, arms, pap...
4799                                                   []
4800    [date, love at first sight, narration, investi...
4801                                                   []
4802            [obsession, camcorder, crush, dream girl]
Name: keywords, Length: 4800, dtype: object

In [30]:
def clean_data(x):
    if isinstance(x, list):
        return [str.lower(i.replace(" ", "")) for i in x]
    else:
        #Check if director exists. If not, return empty string
        if isinstance(x, str):
            return str.lower(x.replace(" ", ""))
        else:
            return ''

In [31]:
features = ['keywords', 'genres']

for feature in features:
    df2[feature] = df2[feature].apply(clean_data)

In [32]:
df2

,id,title,overview,genres,keywords,vote_average,vote_count,year
0,19995,Avatar,"In the 22nd century, a paraplegic Marine is di...","[action, adventure, fantasy, sciencefiction]","[cultureclash, future, spacewar, spacecolony, ...",7.2,11800,2009
1,285,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...","[adventure, fantasy, action]","[ocean, drugabuse, exoticisland, eastindiatrad...",6.9,4500,2007
2,206647,Spectre,A cryptic message from Bond’s past sends him o...,"[action, adventure, crime]","[spy, basedonnovel, secretagent, sequel, mi6, ...",6.3,4466,2015
3,49026,The Dark Knight Rises,Following the death of District Attorney Harve...,"[action, crime, drama, thriller]","[dccomics, crimefighter, terrorist, secretiden...",7.6,9106,2012
4,49529,John Carter,"John Carter is a war-weary, former military ca...","[action, adventure, sciencefiction]","[basedonnovel, mars, medallion, spacetravel, p...",6.1,2124,2012
...,...,...,...,...,...,...,...,...
4798,9367,El Mariachi,El Mariachi just wants to play his guitar and ...,"[action, crime, thriller]","[unitedstates–mexicobarrier, legs, arms, paper...",6.6,238,1992
4799,72766,Newlyweds,A newlywed couple's honeymoon is upended by th...,"[comedy, romance]",[],5.9,5,2011
4800,231617,"Signed, Sealed, Delivered","""Signed, Sealed, Delivered"" introduces a dedic...","[comedy, drama, romance, tvmovie]","[date, loveatfirstsight, narration, investigat...",7.0,6,2013
4801,126186,Shanghai Calling,When ambitious New York attorney Sam is sent t...,[],[],5.7,7,2012


In [33]:
def create_soup(x):
    return ' '.join(x['keywords'])  +x['overview'] + ' '.join(x['genres'])
df2['clean'] = df2.apply(create_soup, axis=1)

In [34]:
new_df = df2[['id', 'title', 'clean']]

In [35]:
new_df['clean'][0]

'cultureclash future spacewar spacecolony society spacetravel futuristic romance space alien tribe alienplanet cgi marine soldier battle loveaffair antiwar powerrelations mindandsoul 3dIn the 22nd century, a paraplegic Marine is dispatched to the moon Pandora on a unique mission, but becomes torn between following orders and protecting an alien civilization.action adventure fantasy sciencefiction'

In [36]:
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
tf = TfidfVectorizer(analyzer='word',ngram_range=(1, 2),min_df=0, stop_words='english')
tfidf_matrix = tf.fit_transform(new_df['clean'])

In [37]:
from sklearn.metrics.pairwise import linear_kernel, cosine_similarity
cosine_sim = cosine_similarity(tfidf_matrix, tfidf_matrix)

In [38]:
cosine_sim[0]

array([1.        , 0.01224911, 0.00702797, ..., 0.00134309, 0.00149473,
       0.        ])

In [39]:
new_df = new_df.reset_index()
titles = new_df['title']
indices = pd.Series(new_df.index, index=new_df['title'])

In [40]:
def get_recommendations(title):
    idx = indices[title] 
    sim_scores = list(enumerate(cosine_sim[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[1:31]
    movie_indices = [i[0] for i in sim_scores]
    return titles.iloc[movie_indices]

In [41]:
get_recommendations('The Dark Knight Rises').head(10)

65                         The Dark Knight
428                         Batman Returns
299                         Batman Forever
1359                                Batman
119                          Batman Begins
2507                             Slow Burn
210                         Batman & Robin
9       Batman v Superman: Dawn of Justice
2193                  Secret in Their Eyes
3818                              Defendor
Name: title, dtype: object

In [42]:
vote_counts = df1[df1['vote_count'].notnull()]['vote_count'].astype('int')
vote_averages = df1[df1['vote_average'].notnull()]['vote_average'].astype('int')
C = vote_averages.mean()
C

5.489140271493213

In [43]:
m = vote_counts.quantile(0.95)
m

5673.149999999998

In [44]:
def weighted_rating(x):
    v = x['vote_count']
    R = x['vote_average']
    return (v/(v+m) * R) + (m/(m+v) * C)

In [45]:
def improved_recommendations(title):
    idx = indices[title]
    sim_scores = list(enumerate(cosine_sim[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[1:26]
    movie_indices = [i[0] for i in sim_scores]
    
    movies_1 = movies.iloc[movie_indices][['id','title', 'vote_count', 'vote_average', 'year']]
    vote_counts = movies_1[movies_1['vote_count'].notnull()]['vote_count'].astype('int')
    vote_averages = movies_1[movies_1['vote_average'].notnull()]['vote_average'].astype('int')
    C = vote_averages.mean()
    m = vote_counts.quantile(0.60)
    qualified = movies_1[(movies_1['vote_count'] >= m) & (movies_1['vote_count'].notnull()) & (movies_1['vote_average'].notnull())]
    qualified['vote_count'] = qualified['vote_count'].astype('int')
    qualified['vote_average'] = qualified['vote_average'].astype('int')
    qualified['wr'] = qualified.apply(weighted_rating, axis=1)
    qualified = qualified.sort_values('wr', ascending=False).head(10)
    return qualified

In [46]:
improved_recommendations('The Dark Knight Rises')

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  from ipykernel import kernelapp as app
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation

,id,title,vote_count,vote_average,year,wr
65,155,The Dark Knight,12002,8,2008,7.194095
119,272,Batman Begins,7359,7,2005,6.342293
1359,268,Batman,2096,7,1989,5.896748
879,22803,Law Abiding Citizen,1486,7,2009,5.802744
428,364,Batman Returns,1673,6,1992,5.605483
813,1924,Superman,1022,6,1978,5.567122
299,414,Batman Forever,1498,5,1995,5.386962
238,98566,Teenage Mutant Ninja Turtles,2636,5,2014,5.333965
9,209112,Batman v Superman: Dawn of Justice,7004,5,2016,5.218895
210,415,Batman & Robin,1418,4,1997,5.191361


In [47]:
df1.head()

,adult,id,title,original_language,overview,genre_ids,popularity,release_date,vote_average,vote_count
0,False,493529,Dungeons & Dragons: Honor Among Thieves,en,A charming thief and a band of unlikely advent...,"[12, 14, 35]",507.373,2023-03-23,7.470,601.0
1,False,758323,The Pope's Exorcist,en,"Father Gabriele Amorth, Chief Exorcist of the ...","[27, 9648, 53]",1798.764,2023-04-05,6.982,228.0
2,False,649609,Renfield,en,Having grown sick and tired of his centuries a...,"[27, 35, 14]",209.828,2023-04-07,7.062,121.0
3,False,420808,Peter Pan & Wendy,en,"Wendy Darling, a young girl afraid to leave he...","[10751, 14, 28, 12]",311.870,2023-04-20,5.981,189.0
4,False,447365,Guardians of the Galaxy Volume 3,en,"Peter Quill, still reeling from the loss of Ga...","[878, 12, 28]",647.851,2023-05-03,8.500,13.0


In [48]:
dummies = pd.get_dummies(df['genre_ids'].apply(pd.Series).stack()).sum(level=0)
dummies.columns = [str(col) for col in dummies.columns]

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:1: FutureWarning: Using the level keyword in DataFrame and Series aggregations is deprecated and will be removed in a future version. Use groupby instead. df.sum(level=1) should use df.groupby(level=1).sum().
  """Entry point for launching an IPython kernel.


In [49]:
len(dummies.columns)

27

In [50]:
df1.shape

(13262, 10)

In [51]:
df.isnull().sum()

adult                       1
backdrop_path            2524
id                          0
title                    6714
original_language        1037
original_title           6714
overview                 1037
poster_path              1291
media_type                  0
genre_ids                1039
popularity                  2
release_date             6716
video                    6716
vote_average             1039
vote_count               1039
name                    13262
original_name           13262
first_air_date          14299
origin_country          14299
gender                  18939
known_for_department    18952
profile_path            19176
known_for               18939
dtype: int64

In [52]:
temp = df[~df.title.isnull()]

In [53]:
temp.isnull().sum()

adult                       0
backdrop_path            1121
id                          0
title                       0
original_language           0
original_title              0
overview                    0
poster_path               139
media_type                  0
genre_ids                   2
popularity                  2
release_date                2
video                       2
vote_average                2
vote_count                  2
name                    13262
original_name           13262
first_air_date          13262
origin_country          13262
gender                  13262
known_for_department    13262
profile_path            13262
known_for               13262
dtype: int64

In [54]:
temp.columns

Index(['adult', 'backdrop_path', 'id', 'title', 'original_language',
       'original_title', 'overview', 'poster_path', 'media_type', 'genre_ids',
       'popularity', 'release_date', 'video', 'vote_average', 'vote_count',
       'name', 'original_name', 'first_air_date', 'origin_country', 'gender',
       'known_for_department', 'profile_path', 'known_for'],
      dtype='object')

In [55]:
temp.drop(["poster_path","original_title","media_type","backdrop_path","video","name",'original_name', 'first_air_date', 'origin_country', 'gender',
       'known_for_department', 'profile_path', 'known_for'],axis=1,inplace=True)


/opt/conda/lib/python3.7/site-packages/pandas/core/frame.py:4913: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [56]:
temp.isnull().sum()

adult                0
id                   0
title                0
original_language    0
overview             0
genre_ids            2
popularity           2
release_date         2
vote_average         2
vote_count           2
dtype: int64

In [57]:
temp.info

<bound method DataFrame.info of        adult      id                                    title  \
0      False  493529  Dungeons & Dragons: Honor Among Thieves   
1      False  758323                      The Pope's Exorcist   
2      False  649609                                 Renfield   
3      False  420808                        Peter Pan & Wendy   
4      False  447365         Guardians of the Galaxy Volume 3   
...      ...     ...                                      ...   
19965  False  456750                          Game Over, Man!   
19970  False  377273                                 Aquarius   
19973  False   21968                                Super Fly   
19974  False  250653                                   E-Team   
19975  False   86226                              Key Witness   

      original_language                                           overview  \
0                    en  A charming thief and a band of unlikely advent...   
1                    en  Father

In [58]:
genre_ids_temp = temp.genre_ids

In [59]:
genre_ids_temp

0               [12, 14, 35]
1             [27, 9648, 53]
2               [27, 35, 14]
3        [10751, 14, 28, 12]
4              [878, 12, 28]
                ...         
19965               [28, 35]
19970                   [18]
19973       [28, 80, 18, 53]
19974                   [99]
19975               [18, 80]
Name: genre_ids, Length: 13262, dtype: object

In [60]:
genre_dict

NameError: name 'genre_dict' is not defined

In [61]:
genre_dict = {
             28
            : "Action"
        ,
        
             12
            : "Adventure"
        ,
        
             16
            : "Animation"
        ,
        
             35
            : "Comedy"
        ,
        
             80
            : "Crime"
        ,
        
             99
            : "Documentary"
        ,
        
             18
            : "Drama"
        ,
        
             10751
            : "Family"
        ,
        
             14
            : "Fantasy"
        ,
        
             36
            : "History"
        ,
        
             27
            : "Horror"
        ,
        
             10402
            : "Music"
        ,
        
             9648
            : "Mystery"
        ,
        
             10749
            : "Romance"
        ,
        
             878
            : "Science Fiction"
        ,
             10770
            : "TV Movie"
        ,
        
             53
            : "Thriller"
        ,
        
             10752
            : "War"
        ,
        
             37
            : "Western"
        
        }

In [62]:
genre_dict

{28: 'Action',
 12: 'Adventure',
 16: 'Animation',
 35: 'Comedy',
 80: 'Crime',
 99: 'Documentary',
 18: 'Drama',
 10751: 'Family',
 14: 'Fantasy',
 36: 'History',
 27: 'Horror',
 10402: 'Music',
 9648: 'Mystery',
 10749: 'Romance',
 878: 'Science Fiction',
 10770: 'TV Movie',
 53: 'Thriller',
 10752: 'War',
 37: 'Western'}

In [67]:
temp['genre_names'] = temp['genre_ids'].apply(lambda x: [genre_dict.get(i) for i in x])

TypeError: 'float' object is not iterable

In [68]:
temp.head()

,adult,id,title,original_language,overview,genre_ids,popularity,release_date,vote_average,vote_count
0,False,493529,Dungeons & Dragons: Honor Among Thieves,en,A charming thief and a band of unlikely advent...,"[12, 14, 35]",507.373,2023-03-23,7.470,601.0
1,False,758323,The Pope's Exorcist,en,"Father Gabriele Amorth, Chief Exorcist of the ...","[27, 9648, 53]",1798.764,2023-04-05,6.982,228.0
2,False,649609,Renfield,en,Having grown sick and tired of his centuries a...,"[27, 35, 14]",209.828,2023-04-07,7.062,121.0
3,False,420808,Peter Pan & Wendy,en,"Wendy Darling, a young girl afraid to leave he...","[10751, 14, 28, 12]",311.870,2023-04-20,5.981,189.0
4,False,447365,Guardians of the Galaxy Volume 3,en,"Peter Quill, still reeling from the loss of Ga...","[878, 12, 28]",647.851,2023-05-03,8.500,13.0


In [69]:
temp.info

<bound method DataFrame.info of        adult      id                                    title  \
0      False  493529  Dungeons & Dragons: Honor Among Thieves   
1      False  758323                      The Pope's Exorcist   
2      False  649609                                 Renfield   
3      False  420808                        Peter Pan & Wendy   
4      False  447365         Guardians of the Galaxy Volume 3   
...      ...     ...                                      ...   
19965  False  456750                          Game Over, Man!   
19970  False  377273                                 Aquarius   
19973  False   21968                                Super Fly   
19974  False  250653                                   E-Team   
19975  False   86226                              Key Witness   

      original_language                                           overview  \
0                    en  A charming thief and a band of unlikely advent...   
1                    en  Father

In [70]:
features = ['genre_names']

for feature in features:
    temp[feature] = temp[feature].apply(clean_data)

KeyError: 'genre_names'

In [71]:
temp.head()

,adult,id,title,original_language,overview,genre_ids,popularity,release_date,vote_average,vote_count
0,False,493529,Dungeons & Dragons: Honor Among Thieves,en,A charming thief and a band of unlikely advent...,"[12, 14, 35]",507.373,2023-03-23,7.470,601.0
1,False,758323,The Pope's Exorcist,en,"Father Gabriele Amorth, Chief Exorcist of the ...","[27, 9648, 53]",1798.764,2023-04-05,6.982,228.0
2,False,649609,Renfield,en,Having grown sick and tired of his centuries a...,"[27, 35, 14]",209.828,2023-04-07,7.062,121.0
3,False,420808,Peter Pan & Wendy,en,"Wendy Darling, a young girl afraid to leave he...","[10751, 14, 28, 12]",311.870,2023-04-20,5.981,189.0
4,False,447365,Guardians of the Galaxy Volume 3,en,"Peter Quill, still reeling from the loss of Ga...","[878, 12, 28]",647.851,2023-05-03,8.500,13.0


In [72]:
temp['overview'] = temp['overview'].apply(lambda x: x.lower())

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [73]:
temp.head()

,adult,id,title,original_language,overview,genre_ids,popularity,release_date,vote_average,vote_count
0,False,493529,Dungeons & Dragons: Honor Among Thieves,en,a charming thief and a band of unlikely advent...,"[12, 14, 35]",507.373,2023-03-23,7.470,601.0
1,False,758323,The Pope's Exorcist,en,"father gabriele amorth, chief exorcist of the ...","[27, 9648, 53]",1798.764,2023-04-05,6.982,228.0
2,False,649609,Renfield,en,having grown sick and tired of his centuries a...,"[27, 35, 14]",209.828,2023-04-07,7.062,121.0
3,False,420808,Peter Pan & Wendy,en,"wendy darling, a young girl afraid to leave he...","[10751, 14, 28, 12]",311.870,2023-04-20,5.981,189.0
4,False,447365,Guardians of the Galaxy Volume 3,en,"peter quill, still reeling from the loss of ga...","[878, 12, 28]",647.851,2023-05-03,8.500,13.0


In [74]:
dummies = pd.get_dummies(df['genre_ids'].apply(pd.Series).stack()).sum(level=0)

dummies.columns = [str(col) for col in dummies.columns]

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:1: FutureWarning: Using the level keyword in DataFrame and Series aggregations is deprecated and will be removed in a future version. Use groupby instead. df.sum(level=1) should use df.groupby(level=1).sum().
  """Entry point for launching an IPython kernel.


In [75]:
dummies.head()

,12.0,14.0,16.0,18.0,27.0,28.0,35.0,36.0,37.0,53.0,...,10752.0,10759.0,10762.0,10763.0,10764.0,10765.0,10766.0,10767.0,10768.0,10770.0
0,1,1,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,1,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
2,0,1,0,0,1,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,1,1,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,1,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [67]:
def create_soup(x):
    return  x['overview'] + ' '.join(x['genre_names'])
temp['clean'] = temp.apply(create_soup, axis=1)

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [68]:
train_1 = temp[['clean','id','title']]

In [69]:
train_1.head()

,clean,id,title
0,"following the latest ghostface killings, the f...",934433,Scream VI
1,salt-of-the-earth cole falls head over heels f...,868759,Ghosted
2,"while working underground to fix a water main,...",502356,The Super Mario Bros. Movie
3,super-hero partners scott lang and hope van dy...,640146,Ant-Man and the Wasp: Quantumania
4,two sisters find an ancient vinyl that gives b...,713704,Evil Dead Rise


In [70]:
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
tf = TfidfVectorizer(analyzer='word',ngram_range=(1, 2),min_df=0, stop_words='english')
tfidf_matrix = tf.fit_transform(train_1['clean'])

In [71]:
from sklearn.metrics.pairwise import linear_kernel, cosine_similarity
cosine_sim = cosine_similarity(tfidf_matrix)

In [72]:
cosine_sim[0]

array([1.        , 0.        , 0.        , ..., 0.00440228, 0.        ,
       0.        ])

In [73]:
train_1 = train_1.reset_index()
titles = train_1['title']
indices = pd.Series(train_1.index, index=train_1['title'])

In [74]:
def get_recommendations(title):
    idx = indices[title] 
    sim_scores = list(enumerate(cosine_sim[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[1:31]
    movie_indices = [i[0] for i in sim_scores]
    return titles.iloc[movie_indices]

In [75]:
get_recommendations('Avatar: The Way of Water').head(10)

7829     Heavy Armor 4: Monster Attack
7108            Dragon Hunting Mystery
2988                     Nobody's Fool
11257                      Night Teeth
4157                        Safe House
8638                      Solar Impact
469                        Terrifier 2
482                        Terrifier 2
11927                    The Two Jakes
4119                           Ayalaan
Name: title, dtype: object

In [76]:
def improved_recommendations(title):
    idx = indices[title]
    sim_scores = list(enumerate(cosine_sim[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[1:26]
    movie_indices = [i[0] for i in sim_scores]
    
    movies_1 = temp.iloc[movie_indices][['id','title', 'vote_count', 'vote_average']]
    vote_counts = movies_1[movies_1['vote_count'].notnull()]['vote_count'].astype('int')
    vote_averages = movies_1[movies_1['vote_average'].notnull()]['vote_average'].astype('int')
    C = vote_averages.mean()
    m = vote_counts.quantile(0.60)
    qualified = movies_1[(movies_1['vote_count'] >= m) & (movies_1['vote_count'].notnull()) & (movies_1['vote_average'].notnull())]
    qualified['vote_count'] = qualified['vote_count'].astype('int')
    qualified['vote_average'] = qualified['vote_average'].astype('int')
    qualified['wr'] = qualified.apply(weighted_rating, axis=1)
    qualified = qualified.sort_values('wr', ascending=False).head(10)
    return qualified

In [77]:
improved_recommendations('Avatar: The Way of Water')

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  from ipykernel import kernelapp as app
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation

,id,title,vote_count,vote_average,wr
1219,119450,Dawn of the Planet of the Apes,10067,7,6.476598
6051,59961,Safe House,3181,6,5.702214
6765,8456,Never Back Down,2078,6,5.659329
13106,7305,Alive,1247,6,5.617867
728,663712,Terrifier 2,1158,6,5.612821
710,663712,Terrifier 2,1157,6,5.612763
16788,669671,Night Teeth,647,6,5.581053
4325,11593,Nobody's Fool,248,6,5.552376
12367,13613,Day of the Dead,323,5,5.503353
6377,10069,Stay Alive,636,5,5.478012


In [78]:
temp.head()

,adult,id,title,original_language,overview,genre_ids,popularity,release_date,vote_average,vote_count,genre_names,clean
0,False,934433,Scream VI,en,"following the latest ghostface killings, the f...","[27, 9648, 53]",311.194,2023-03-08,7.318,567.0,"[horror, mystery, thriller]","following the latest ghostface killings, the f..."
1,False,868759,Ghosted,en,salt-of-the-earth cole falls head over heels f...,"[10749, 28, 35]",316.240,2023-04-21,7.275,318.0,"[romance, action, comedy]",salt-of-the-earth cole falls head over heels f...
2,False,502356,The Super Mario Bros. Movie,en,"while working underground to fix a water main,...","[16, 12, 10751, 14, 35]",10478.582,2023-04-05,7.508,1387.0,"[animation, adventure, family, fantasy, comedy]","while working underground to fix a water main,..."
3,False,640146,Ant-Man and the Wasp: Quantumania,en,super-hero partners scott lang and hope van dy...,"[28, 12, 878]",12579.923,2023-02-15,6.512,1796.0,"[action, adventure, sciencefiction]",super-hero partners scott lang and hope van dy...
4,False,713704,Evil Dead Rise,en,two sisters find an ancient vinyl that gives b...,"[27, 53]",2703.373,2023-04-12,7.000,170.0,"[horror, thriller]",two sisters find an ancient vinyl that gives b...


In [80]:
temp1 = temp[['title','clean']]

In [81]:
temp1

,title,clean
0,Scream VI,"following the latest ghostface killings, the f..."
1,Ghosted,salt-of-the-earth cole falls head over heels f...
2,The Super Mario Bros. Movie,"while working underground to fix a water main,..."
3,Ant-Man and the Wasp: Quantumania,super-hero partners scott lang and hope van dy...
4,Evil Dead Rise,two sisters find an ancient vinyl that gives b...
...,...,...
19970,Fast Food,auggie wants to make it big and has tried pret...
19971,County Hospital,ollie is in the hospital with a broken leg. wh...
19972,Limbo,"a murderer finds himself on trial in hell, cau..."
19973,The Libertine,a sexy widow discovers her late husband had a ...


In [82]:
import pickle
pickle.dump(temp1.to_dict(), open('movie_dict.pkl', 'wb'))

In [83]:
import pickle
pickle.dump(cosine_sim, open('cosine_sim.pkl', 'wb'))